# Creating Surveys
This notebook contains code for creating questions and combining them into `Survey` objects in EDSL, with examples of ways to add question memory, stop logic and other conditional rules.

For more details on the `Survey` class and methods, see the <a href="https://docs.expectedparrot.com/en/latest/surveys.html">Surveys section</a> of the docs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/create_surveys.ipynb)

In [1]:
# Install the edsl package 
# ! pip install edsl

In [1]:
# Import the tools
from edsl.questions import QuestionLinearScale, QuestionCheckBox, QuestionYesNo, QuestionMultipleChoice, QuestionFunctional
from edsl.questions.compose_questions import compose_questions
from edsl import Survey

In [6]:
# Create some example questions
q1 = QuestionYesNo(
    question_name = "fit",
    question_text = "Do you find it difficult to shop for clothes that fit you?"
) 

q2 = QuestionMultipleChoice(
    question_name = "online",
    question_text = "How often do you shop for clothes online?",
    question_options = ["Never", "Rarely", "Occasionally", "Often"]
)

Survey(questions=[QuestionYesNo(question_name = 'fit', question_text = 'Do you find it difficult to shop for clothes that fit you?', question_options = ['Yes', 'No'], short_names_dict = {}), QuestionMultipleChoice(question_name = 'online', question_text = 'How often do you shop for clothes online?', question_options = ['Never', 'Rarely', 'Occasionally', 'Often'], short_names_dict = {})], name=None)

In [ ]:
# Questions can be combined in multiple ways
# Method 1: add questions to a Survey object
Survey(questions = [q1, q2])

In [ ]:
# Method 2: add_question() chaining 
q1.add_question(q2)

In [13]:
# We can seed questions to refer to other questions and answers

q_eat = QuestionYesNo(
    question_name = "eat",
    question_text = "Do you like to eat {{ food }}?"
)

q_sam = QuestionYesNo(
    question_name = "sam",
    question_text = "Are you Sam I Am?"
)

survey = Survey(questions = [q_eat, q_sam])

# Method 1: add_targeted_memory()
survey.add_targeted_memory(q_sam, q_eat)

results = survey.by(Scenario({"food":"green eggs and ham"})).run()

We can inspect the `_user_prompt` field of the second question to see that it has been modified to include the first question and response:

In [14]:
results.select("prompt.*").print()

prompt.sam_system_prompt,prompt.eat_user_prompt,prompt.eat_system_prompt,prompt.sam_user_prompt
"{'text': 'You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{}', 'class_name': 'AgentInstruction'}","{'text': 'You are being asked the following question: Do you like to eat green eggs and ham?\nThe options are\n\n0: Yes\n\n1: No\n\nReturn a valid JSON formatted like this, selecting only the number of the option:\n{""answer"": , ""comment"": """"}\nOnly 1 option may be selected.', 'class_name': 'YesNo'}","{'text': 'You are answering questions as if you were a human. Do not break character. You are an agent with the following persona:\n{}', 'class_name': 'AgentInstruction'}","{'text': 'You are being asked the following question: Are you Sam I Am?\nThe options are\n\n0: Yes\n\n1: No\n\nReturn a valid JSON formatted like this, selecting only the number of the option:\n{""answer"": , ""comment"": """"}\nOnly 1 option may be selected.\n Before the question you are now answering, you already answered the following question(s):\n \tQuestion: Do you like to eat green eggs and ham?\n\tAnswer: No', 'class_name': 'YesNo'}"


### Method 2: Using the `compose_questions()` method
You can also use the `compose_questions()` method to include the response to one question in the context of another question:

In [15]:
q_eat = QuestionYesNo(
    question_name = "eat",
    question_text = "Do you like to eat {{ food }}?"
)

q_sam = QuestionYesNo(
    question_name = "sam",
    question_text = "You were previously asked: '" + q_eat.question_text 
        + "' You responded: '{{ food }}'. Are you Sam I Am?"
)

q_eat_sam = compose_questions(q_eat, q_sam).by(Scenario({"food":"green eggs and ham"}))

In [16]:
q_eat_sam.print()

                               Jobs                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Jobs                                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Questions                                                   ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │ │
│ │ ┃ Question Name ┃ Question Type ┃ Question Text ┃ Options ┃ │ │
│ │ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │ │
│ │ │ eat_sam       │ functional    │ functional    │ None    │ │ │
│ │ └───────────────┴───────────────┴───────────────┴─────────┘ │ │
│ └─────────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────────┘

## Survey rules
We can add rules to a survey, such as skip logic and stop logic. Here we at a stop rule to end the survey at the first question if the response is "No":

In [17]:
survey_eat_sam = (q_eat
                 .add_question(q_sam)
                 .add_stop_rule("eat", "eat == 'no'")
)

In [18]:
survey_eat_sam.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓                                      │
│ ┃ Question Name ┃ Question Type ┃ Question Text                ┃ Options ┃                                      │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩                                      │
│ │ eat           │ yes_no        │ Do you like to eat {{food}}? │ Yes, No │                                      │
│ └───────────────┴───────────────┴──────────────────────────────┴─────────┘                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ sam           │ yes_no        │ You were previously asked: 'Do you like to eat {{food}}?' You     │ Yes, No │ │
│ │               │               │ responded: '{{food}}'. Are you Sam I Am?                          │         │ │
│ └───────────────┴───────────────┴───────────────────────────────────────────────────────────────────┴─────────┘ │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

Note that adding the questions creates a Survey, to which we append the stop rule. We can also explicitly create a Survey with the stop rule (these are equivalent):

In [19]:
survey_eat_sam_v2 = Survey([q_eat, q_sam]).add_stop_rule("eat", "eat == 'no'")

In [20]:
survey_eat_sam_v2.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓                                      │
│ ┃ Question Name ┃ Question Type ┃ Question Text                ┃ Options ┃                                      │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩                                      │
│ │ eat           │ yes_no        │ Do you like to eat {{food}}? │ Yes, No │                                      │
│ └───────────────┴───────────────┴──────────────────────────────┴─────────┘                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ sam           │ yes_no        │ You were previously asked: 'Do you like to eat {{food}}?' You     │ Yes, No │ │
│ │               │               │ responded: '{{food}}'. Are you Sam I Am?                          │         │ │
│ └───────────────┴───────────────┴───────────────────────────────────────────────────────────────────┴─────────┘ │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [21]:
from edsl import Agent 

agents = [Agent(traits={"persona":p}) for p in ["You are Sam I am.", "You are John Horton.", "You are a vegan."]]

scenario = Scenario({"food":"green eggs and ham"})

results = survey_eat_sam.by(scenario).by(agents).run()

In [22]:
(results
.select("persona", "scenario.*", "eat", "sam")
.print(pretty_labels={"agent.persona":"Persona", "scenario.food":"Food", "answer.eat":q_eat.question_text, "answer.sam":"Are you Sam I Am?"})
)

Persona,Food,Do you like to eat {{food}}?,Are you Sam I Am?
You are Sam I am.,green eggs and ham,Yes,Yes
You are John Horton.,green eggs and ham,No,No
You are a vegan.,green eggs and ham,No,No
